In [ ]:
%load_ext kamu

In [ ]:
%%local
import pandas as pd
import hvplot.pandas
#pd.set_option('display.max_rows', 500)
#pd.set_option('display.max_columns', 500)
#pd.set_option('display.width', 1000)
pd.set_option('max_colwidth', 1000)

In [ ]:
%import_dataset matches
%import_dataset player-scores
%import_dataset player-scores-total

In [ ]:
%%sql -o scores
select * from `player-scores`

In [ ]:
%%local
scores.hvplot.box("score", by="name", height=500, width=800, title="Players score distribution")